# 4.1. - Redes GRU y LSTM

En este notebook, exploraremos dos tipos de Redes Neuronales Recurrentes (RNN): las Long Short-Term Memory (LSTM) y las Gated Recurrent Unit (GRU). Ambas están diseñadas para manejar **secuencias de datos** y resolver el problema del **desvanecimiento del gradiente** de las redes recurrentes básicas.

## 4.1.1. - Repaso de teoría

### Redes LSTM

Las LSTM fueron introducidas por Hochreiter y Schmidhuber en 1997. La clave de las LSTM es la celda de memoria, que permite a la red recordar valores por largos periodos de tiempo.

#### Estructura de una LSTM

Una LSTM consta de una celda de memoria, una puerta de entrada, una puerta de salida y una puerta de olvido. Cada una de estas puertas tiene una función específica para controlar el flujo de información dentro de la celda de memoria.

### Redes GRU

Las GRU fueron introducidas por Cho et al. en 2014. Son una variante simplificada de las LSTM, con menos puertas. Consta de dos puertas principales: una puerta de actualización y una puerta de reinicio.

## 4.1.2. - Conjunto de datos
Para este ejemplo, utilizaremos un pequeño conjunto de datos de texto (La venganza de Don Mendo) para entrenar los modelos GRU y LSTM. La tarea consistirá en predecir la siguiente palabra en una secuencia.


In [4]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import requests
import re
from collections import Counter
from torch.utils.data import Dataset, DataLoader, random_split

# Descargar el texto de "La venganza de Don Mendo" de Pedro Muñoz Seca
url = "https://www.gutenberg.org/cache/epub/49013/pg49013.txt"
response = requests.get(url, timeout=30)
text = response.text

# Preprocesar el texto
text = text.lower()
text = re.sub(r'[^a-z\s]', '', text)
words = text.split()

# Crear diccionarios de palabras a índices y viceversa
word_counts = Counter(words)
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}
vocab_size = len(vocab)

# Crear el dataset
class TextDataset(Dataset):
    def __init__(self, words, word_to_ix, seq_length):
        self.data = []
        self.seq_length = seq_length
        for i in range(len(words) - seq_length):
            seq_in = words[i:i + seq_length]
            seq_out = words[i + seq_length]
            self.data.append((seq_in, seq_out))
        self.word_to_ix = word_to_ix

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq_in, seq_out = self.data[idx]
        seq_in = torch.tensor([self.word_to_ix[word] for word in seq_in], dtype=torch.long)
        seq_out = torch.tensor(self.word_to_ix[seq_out], dtype=torch.long)
        return seq_in, seq_out

seq_length = 5
dataset = TextDataset(words, word_to_ix, seq_length)

# Dividir el dataset en entrenamiento, validación y prueba
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [5]:
# Calcular el peso para cada palabra
total_words = len(words)
word_freq = np.array([word_counts[word] for word in vocab])
inverse_freq = total_words / (word_freq + 1e-10)  # Añadir un pequeño valor para evitar división por cero
weights = np.max(inverse_freq) / inverse_freq  # Normalizar los pesos

## 4.1.3. - Modelos


In [6]:
# Definir el modelo GRU
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        x = self.embedding(x)
        out, h = self.gru(x, h)
        out = self.fc(out[:, -1, :])
        return out, h

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

# Definir el modelo LSTM
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, h):
        x = self.embedding(x)
        out, (h, c) = self.lstm(x, h)
        out = self.fc(out[:, -1, :])
        return out, (h, c)

    def init_hidden(self, batch_size):
        return (torch.zeros(1, batch_size, self.hidden_size),
                torch.zeros(1, batch_size, self.hidden_size))


In [7]:
import torch.nn.functional as F

# Definir la función de pérdida con pesos
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, weights):
        super(WeightedCrossEntropyLoss, self).__init__()
        self.weights = weights
    
    def forward(self, outputs, targets):
        # Calcular la pérdida cruzada con los pesos
        return F.cross_entropy(outputs, targets, weight=self.weights)
    
# Convertir los pesos a un tensor de PyTorch
weights = torch.tensor(weights, dtype=torch.float32)

In [14]:
# Parámetros del modelo
embed_size = 256
hidden_size = 128
output_size = vocab_size
num_epochs = 20
learning_rate = 0.0001

# Inicializar los modelos, loss function y optimizer
gru_model = GRUModel(vocab_size, embed_size, hidden_size, output_size)
lstm_model = LSTMModel(vocab_size, embed_size, hidden_size, output_size)
criterion = WeightedCrossEntropyLoss(weights) # nn.CrossEntropyLoss()
gru_optimizer = optim.Adam(gru_model.parameters(), lr=learning_rate)
lstm_optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)

## 4.1.4. - Entrenamiento

In [15]:
# Función de entrenamiento y validación
def train_model(model, optimizer, train_loader, val_loader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_train_loss = 0
        total_val_loss = 0

        # Entrenamiento
        for inputs, targets in train_loader:
            h = model.init_hidden(inputs.size(0))
            optimizer.zero_grad()
            outputs, _ = model(inputs, h)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # Validación
        model.eval()
        with torch.no_grad():
            for inputs, targets in val_loader:
                h = model.init_hidden(inputs.size(0))
                outputs, _ = model(inputs, h)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
        model.train()

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {total_train_loss/len(train_loader):.4f}, '
              f'Val Loss: {total_val_loss/len(val_loader):.4f}')

# Entrenar el modelo GRU
print("Entrenando modelo GRU...")
train_model(gru_model, gru_optimizer, train_loader, val_loader, num_epochs)

# Entrenar el modelo LSTM
print("Entrenando modelo LSTM...")
train_model(lstm_model, lstm_optimizer, train_loader, val_loader, num_epochs)


Entrenando modelo GRU...
Epoch [1/20], Train Loss: 8.3835, Val Loss: 8.1756
Epoch [2/20], Train Loss: 7.4496, Val Loss: 5.9779
Epoch [3/20], Train Loss: 4.2070, Val Loss: 3.4668
Epoch [4/20], Train Loss: 3.2983, Val Loss: 3.3115
Epoch [5/20], Train Loss: 3.2039, Val Loss: 3.2717
Epoch [6/20], Train Loss: 3.1723, Val Loss: 3.2433
Epoch [7/20], Train Loss: 3.1298, Val Loss: 3.2194
Epoch [8/20], Train Loss: 3.0962, Val Loss: 3.1986
Epoch [9/20], Train Loss: 3.0664, Val Loss: 3.1760
Epoch [10/20], Train Loss: 3.0342, Val Loss: 3.1561
Epoch [11/20], Train Loss: 3.0015, Val Loss: 3.1359
Epoch [12/20], Train Loss: 2.9683, Val Loss: 3.1158
Epoch [13/20], Train Loss: 2.9268, Val Loss: 3.0968
Epoch [14/20], Train Loss: 2.8994, Val Loss: 3.0787
Epoch [15/20], Train Loss: 2.8576, Val Loss: 3.0600
Epoch [16/20], Train Loss: 2.8214, Val Loss: 3.0436
Epoch [17/20], Train Loss: 2.7862, Val Loss: 3.0284
Epoch [18/20], Train Loss: 2.7466, Val Loss: 3.0135
Epoch [19/20], Train Loss: 2.7129, Val Loss: 3.0

KeyboardInterrupt: 

In [16]:
# Función de predicción
def predict(model, word_to_ix, ix_to_word, start_words, predict_len):
    model.eval()
    input_seq = torch.tensor([word_to_ix[word] for word in start_words], dtype=torch.long).unsqueeze(0)
    h = model.init_hidden(input_seq.size(0))
    predicted_words = start_words

    for _ in range(predict_len):
        output, h = model(input_seq, h)
        _, top_idx = torch.max(output, 1)
        predicted_word = ix_to_word[top_idx.item()]
        predicted_words.append(predicted_word)
        input_seq = torch.cat((input_seq[:, 1:], top_idx.unsqueeze(0)), 1)

    return ' '.join(predicted_words)

# Predecir con el modelo GRU
print("Predicción con GRU:")
print(predict(gru_model, word_to_ix, ix_to_word, ["si", "desenvaino", "el", "acero", "vais"], 10))

# Predecir con el modelo LSTM
print("Predicción con LSTM:")
print(predict(lstm_model, word_to_ix, ix_to_word, ["si", "desenvaino", "el", "acero", "vais"], 10))

Predicción con GRU:
si desenvaino el acero vais de la de la de la de la de la
Predicción con LSTM:
si desenvaino el acero vais de de de de de de de de de de
